Main sheet: https://docs.google.com/spreadsheets/d/19gxxCEdUL0FdEU4B_IC2D5XaOzjxCtQC0Su13lCVOGQ/edit?usp=sharing

Last Week: Finished Genus Description, Genus, Epithet (still errors)
Monday, Tuesday: Finished floraison, locality (still errors with locality of genus description)

Wednesday:
To Do:
- Fix errors in helper functions (str(fp) instead of str(font)) and see if this fixes epithet
- finish aire geogr
- finish notes
- make a new cleaner notebook
Accomplished:
- Figured out to use bbox to determine spacing between text
- Added spreadsheet to the Analyzed sheet (this time for over 100 pages) to be able to analyze line heights

Monday (9/21):
To Do:
- Fix errors in helper functions (str(fp) instead of str(font)) and see if this fixes epithet
- finish aire geogr
- finish notes
- Remove Novelle Flore
- make a new cleaner notebook
Accomplished:
- wrote the necessary identification functions
- discovered weird edge cases (in analyzed comprehensive)
- Geography works for the most part just not notes

Questions for Aaron
- Why is this line not working?: if prev_height > box[1] and abs(box[0] - indent) < 1:

Monday (9/28):
- Find what is causing errors
- find out why nothing is going into the notes when it should
- finish last few cols

Next week:
- finish rest of columns

In [23]:
!pip install pyresparser fitz PyMuPDF pypdf2 >/dev/null

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [42]:
!ln -sf "NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf" document.pdf #alias

In [43]:
import fitz
document = 'document.pdf'
filename = document
search_term = "the"
pdf_document = fitz.open(filename)

In [44]:
doc = pdf_document
dir(doc)

['FontInfos',
 'FormFonts',
 'Graftmaps',
 'PDFCatalog',
 'PDFTrailer',
 'ShownPages',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 '_addFormFont',
 '_delToC',
 '_delXmlMetadata',
 '_deleteObject',
 '_deletePage',
 '_do_links',
 '_dropOutline',
 '_embeddedFileAdd',
 '_embeddedFileDel',
 '_embeddedFileGet',
 '_embeddedFileIndex',
 '_embeddedFileInfo',
 '_embeddedFileNames',
 '_embeddedFileUpd',
 '_forget_page',
 '_getCharWidths',
 '_getMetadata',
 '_getNewXref',
 '_getOLRootNumber',
 '_getPDFfileid',
 '_getPDFroot',
 '_getPageInfo',
 '_getPageObjNumber',
 '_getPageXref',
 '_getTra

In [45]:
# https://pymupdf.readthedocs.io/en/latest/faq/#how-to-analyze-font-characteristics

import fitz
import pprint, re

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)


doc = fitz.open("document.pdf")
page = doc[528]

In [46]:
#start of my code

In [47]:
import pandas as pd #imports the library pandas

In [48]:
#creates a list of the different columns
columns = ['Family',
 'GenusDescription',
 'Genus',
 'epithet',
 'author',
 'syn.',
 'Plate',
 'description',
 'FloraisonStart',
 'FloraisonEnd',
 'FloraisonNotes',
 'L.',
 'S.',
 'Aire géogr.',
 'Notes']

In [49]:
#defines the Face object
class Face:
    def __init__(self, font, size, color, flags=0):
        self.font = font
        self.size = size
        self.color = color
        self.flags = flags
        self.style = flags_decomposer(self.flags)
    def __str__(self):
        return f"Font: '{self.font}' ({self.style}), size {self.size:g}, color " + ("#%06x" % self.color)

#checks if the following text is part of the family column
#t - text
#fp - face object
def isfamily(t, fp):
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    if  re.match('^[A-Z]+$', t) or font == str(fp):
        return True

#checks if the following text is part of the genus description column
#t - text
#fp - face object
def isGenGenus(t, fp):
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    if re.match('[A-Z]+\s[A-Z][a-z]*\.\s\—\s[A-Z]([a-z]|\s|,|[a-zàâçéèêëîïôûùüÿñæœ .-])+', t) or str(font) == fp:
        #print(f'''Text: {t}\n{fp}\n\n''')
        return True
    
#checks if the following text is a continuation of the genus description column
#t - text
#fp - face object
#prevTest - previous text
def isGenGenusContinued(t, fp, prevText):
    #'Times-Roman' (serifed, proportional), size 11, color #000000
#     print("fon", fp)
#     print("text", t)
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    #re.match('([A-Z][a-z]|\s|,|[a-zàâçéèêëîïôûùüÿñæœ .-])+', t)  
#     print("prev",prevText)
    if font != str(fp) and prevText:
        #print(f'''Text: {t}\n{fp}\n\n''')
        return False
    return True

#checks if the following text ends with a period
#t - text
def prevHasPeriod(t):
    return re.match('.*(\.)$', t)

#checks if the following text is part of the genus column
#t - text
#fp - face object
def isGenus(t, fp):
    font = "Font: 'Times-Bold' (serifed, proportional, bold), size 10.9449, color #000000"
    font2 = "Font: 'Times-Bold' (serifed, proportional, bold), size 11, color #000000"
    font3 = "Font: 'VHVHMH+Times-Bold' (serifed, proportional, bold), size 10.8833, color #000000"
    font4 = "Font: 'VHVHMH+Times-Bold' (serifed, proportional, bold), size 10.8403, color #000000"
    if (re.match('[A-Z][a-z]+', t) or re.match('[A-Z][a-z]+\s[a-z]+', t)) and (font == str(fp) or font2 == str(fp) or font3 == str(fp) or font4 == str(fp)):
        return True

#checks if the following text is part of the epithet column
#t - text
#fp - face object
def isEpithet(t, fp):
    font = "Font: 'Times-Bold' (serifed, proportional, bold), size 11, color #000000"
    font2 = "Font: 'Times-Bold' (serifed, proportional, bold), size 10.9449, color #000000"
    font3 = "Font: 'VHVHMH+Times-Bold' (serifed, proportional, bold), size 10.8833, color #000000"
    font4 = "Font: 'VHVHMH+Times-Bold' (serifed, proportional, bold), size 10.8403, color #000000"
    if re.match('\s[a-z]+', t) and (font == str(fp) or font2 == str(fp) or font3 == str(fp) or font4 == str(fp)):
        #print(f'''Text: {t}\n{fp}\n\n''')
        return True

#checks if the following text is part of the author column
#t - text
#fp - face object
def isAuthor(t, fp):
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    if str(font) == fp:
        #print(f'''Text: {t}\n{fp}\n\n''')
        return True

#checks if the following text is part of the florison column
#t - text
#fp - face object
def isFlori(t, fp):
    return re.match('Floraison:.*', t)

#checks if the following text is part of the L column
#t - text
#fp - face object
def isL(t,fp):
    #genus description
    font = "Font: 'Times-Italic' (italic, serifed, proportional), size 8.95489, color #000000"
    #some species
    font2 = "Font: 'Times-Roman' (serifed, proportional), size 8.95489, color #000000"
    #most species
    font3 = "Font: 'Times-Roman' (serifed, proportional), size 9.94987, color #000000"
    return re.match('L\..*', t) and (font == str(fp) or font2 == str(fp) or font3 == str(fp))

#checks if the following text is part of the S column
#t - text
#fp - face object
def isS(t, fp):
    #genus description
    font = "Font: 'Times-Italic' (italic, serifed, proportional), size 8.95489, color #000000"
    #some species
    font2 = "Font: 'Times-Roman' (serifed, proportional), size 8.95489, color #000000"
    #most species
    font3 = "Font: 'Times-Roman' (serifed, proportional), size 9.94987, color #000000"
    return re.match('S\..*', t) and (font == str(fp) or font2 == str(fp) or font3 == str(fp))

#checks if the following text is part of the geography column
#t - text
#fp - face object
def isGeo(t, fp):
    return re.match('Aire géogr\. —.*', t)

#in the works
#checks if the following text is part of the page header ("NOVELLE FLORE")
#t - text
#fp - face object
def isHeader(t):
    return re.match('NOUVELLE FLORE', t)

#in the works
#checks if the following text is a page number
#t - text
#fp - face object
def isPageNum(t):
    return re.match("^[0-9]*$", t)

In [50]:
#creates data frame to store our data
frame = pd.DataFrame(columns = columns)
#set first row in frame to 0s
frame.loc[len(frame)] = 0

#initializes all conditional variables (that help with the identification functions above as their default)
onGenGenus = False
floriCount = 0
onGenus = False
onL = False
onS = False
skip = False
onGeo = False
onNotes = False

#helper function which returns conditional variables back to their default settings
def off():
    global onGenGenus
    global floriCount
    global onGenus
    global onL
    global onS
    global skip
    global onGeo
    global onNotes
    
    onGenGenus = False
    floriCount = 0
    onGenus = False
    onL = False
    onS = False
    skip = False
    onGeo = False
    onNotes = False

#line below only used for testing (ignore)
df = pd.DataFrame(columns = ["page","text", "face", "h1", "h2", "l", "r"])
#iterates through pages in specified range
for page in (doc[x] for x in range(8,554)):

    # read page text as a dictionary, suppressing extra spaces in CJK fonts
    blocks = page.getText("dict", flags=11)["blocks"]

    for x,b in enumerate(blocks): # iterate through the text blocks
        for y,l in enumerate(b["lines"]):  # iterate through the text lines
            for z,s in enumerate(l["spans"]):  # iterate through the text spans
                
                s["style"] = flags_decomposer(s["flags"])  # readable font flags
                face = Face(s["font"], s["size"], s["color"], s["flags"])
                #(x0, y0, x1, y1) top left corner to bottom right corner
                box = s["bbox"]
                
######for testing purposes (ignore)
#                 df.loc[len(df)] = 0
#                 df.iloc[-1]['text'] = s["text"]
#                 df.iloc[-1]['face'] = str(face)
#                 df.iloc[-1]["h1"] = box[1]
#                 df.iloc[-1]["h2"] = box[3]
#                 df.iloc[-1]["l"] = box[0]
#                 df.iloc[-1]["r"] = box[2]
#                 df.iloc[-1]["page"] = page.number
#######
                
########creating a data frame for analyzed
    #             values = [s['text'],s['font'], s['size'], s['style'], s['color'], s['flags']]
    #             columns = ['text', 'font', 'size', 'style', 'color', 'flags']
    #             zipped = zip(columns, values)
    #             a_dictionary = dict(zipped)
    #             frame = frame.append(a_dictionary,ignore_index=True)
    #             print(frame)
    #             print(s['text'], "***",s['font'], "***", s['size'], "***", s['style'], "***", s['color'], "***", s['flags'],"********************************************************\n")
#                 print(onS)
#                 print(s['text'])
#                 if s["text"] == "NOUVELLE FLORE" and 


#                 if skip:
#                     print(s["text"])
#                     skip = False
#                     continue
#                 elif prev_height - box[1] > 500 and last :
#                     print(s["text"])
#                     skip = True
#                     continue
#########
#########attepmt at removing headers on each page
#                 if skip:
#                     print(s["text"])
#                     skip = False
#                     continue
#                 if page.number%2 == 0 and isHeader(s["text"]):
#                     if z + 1 == len(l["spans"]):
#                         if isPageNum(b["lines"][y + 1]["spans"][0]["text"]):
#                             skip = True
#                             continue
#                     else:
#                         if isPageNum(l["spans"][z + 1]["text"]):
#                             skip = True
#                             continue
#                 elif page.number%2 == 0 and isPageNum(s["text"]):
#                     print(s["text"])
#                     if z + 1 == len(l["spans"]):
#                         if isHeader(b["lines"][y + 1]["spans"][0]["text"]):
#                             skip = True
#                             continue
#                     else:
                        
#                         if isHeader(l["spans"][z + 1]["text"]):
#                             skip = True
#                             continue
                            
#                 if isHeader(s["text"], face) or isPageNum(s["text"], face):
#                     continue
#########
                if onGenGenus: #if genGenus function was previously satisfied check to see if lines following are also under Genus Description
                    if isGenGenusContinued(s["text"], face, prevPeriod): #if this line is a continuation of Genus Description
                        frame.iloc[-1]['GenusDescription'] += s["text"] #add the text to the Genus Description column
                        prevPeriod = prevHasPeriod(s["text"]) #checks if sentence has period to set prevPeriod to correct value for next line
                    else:
                        onGenGenus = False #line is no longer under Genus Description so set onGenGenus to False

                if isfamily(s["text"], face) and len(l["spans"]) == 1:
#                     print("family")
#                     print(x, len(blocks), y, z, s["text"], face)
    #             elif isfamily(s["text"], face) and len(l["spans"]) == 1:
    #                 item['Family'] = s["text"]
    #                 print(x, len(blocks), y, z, s["text"], face)
    #                 break
                    continue #replace since code has not been implemented yet

                elif isGenGenus(s["text"], face): #true if belongs in the column Genus Description
                    off() #since this is the first column, sets resets all conditional values
                    onGenGenus = True #turns onGenGenus to true so that next piece of text can be evaluated to see if it also belongs in genGenus
                    if not re.match('.*(\.)$', s['text']): #checks for period
                        prevPeriod = False
                    else:
                        prevPeriod = True
#                     print("description")
#                     print(x, len(blocks), y, z, s["text"], face)
#                     arr.append([s['text'], page])
                    frame.loc[len(frame)] = 0 #creates new roles initialized to 0s
                    frame.iloc[-1]['GenusDescription'] = s["text"] #adds text to GenusDescription column

                elif isGenus(s["text"], face): #true if belongs in Genus
                    off() #resets all conditional values
                    frame.loc[len(frame)] = 0 #creates a new row initialized to 0s
                    if re.match(".*\s.*", s["text"]): #checks if Genus and epithet are on one line or two
                        frame.iloc[-1]['Genus'] = s["text"].split()[0] #splits them if they are on one line
                        frame.iloc[-1]['epithet'] = s["text"].split()[1]
                    else:
                        frame.iloc[-1]['Genus'] = s["text"].split()[0] #otherwise only takes in first word
                        onGenus = True #checks next text for epithet
                elif onGenus: #true if epithet in current line
                    frame.iloc[-1]['epithet'] = s["text"] #adds text to epithet column
                    onGenus = False #switches onGenus off
                elif onNotes: #true if text is a continuation of notes
                    frame.iloc[-1]['Notes'] += s["text"] #add text onto notes
                elif onGeo: #true if previous text was part of Geography
                    if isHeader(s["text"]) or isPageNum(s["text"]): #if text is header or page number pass over it
                        continue
#                    if prev_height > box[1] and abs(box[0] - indent) < 1: #this line errors (initialize prev_height), purpose is to 
                        onGeo = False
                        frame.iloc[-1]['Notes'] = s["text"] #move onto notes
                        onNotes = True
                    else:
                        prevHeight = box[1]
                        frame.iloc[-1]['Aire géogr.'] += s["text"] #true if text belongs to geography column (in the works)
                elif isGeo(s["text"], face): #true if text belongs to geography column
                    off() #resets all initial conditions
                    frame.iloc[-1]['Aire géogr.'] = s['text'] #makes geography the last row
                    onGeo = True
                    prevHeight = box[1] #tracks height
                    indent = box[0] #tracks indent
                elif isL(s["text"], face): #true if belongs to column L
                    off() #resets all initial conditions
                    floriCount = 0 #resets floricount
                    frame.iloc[-1]['L.'] = s["text"] #adds text to L column
                    onL = True #turns on onL
                elif isS(s["text"], face): #true if belongs to column S
                    off() #resets all initial conditions
                    frame.iloc[-1]['S.'] = s["text"] #adds text to S column
                    onS = True #turns on onS
                elif onL: #True if previous text belongs to column L
                    frame.iloc[-1]['L.'] += s["text"] #adds text to column L
                elif onS: #True if previous text belongs to column S
                    frame.iloc[-1]['S.'] += s["text"] #adds text to column S
                
                elif floriCount: #if floriCount greater than 1, floricount is basically the number of parts to flori there are
                    if isL(s["text"], face) or isS(s["text"], face): #exit if belongs to other column
                        floriCount = 0
#                     else:
#                         frame.iloc[-1]['Floraison'] += s["text"]
                    else:
                        lis = s["text"].split()
#                         print(lis)
                        sent = s["text"]
                        if floriCount == 1:
                            if "-" in lis[0]: # splits dates
                                months = lis[0].split("-")
                                frame.iloc[-1]['FloraisonStart'] = months[0]
                                frame.iloc[-1]['FloraisonEnd'] = months[1][:-1]
                                lis = lis[1:]
                                sent = sent[sent.find(" ") + 1:]
                            floriCount += 1
                            if lis:
                                frame.iloc[-1]['FloraisonNotes'] = sent
                                lis = lis[1:]
                                floriCount += 1
                        else:
                            frame.iloc[-1]['FloraisonNotes'] = sent #adds rest of text to Floraison Notes
                            floriCount += 1
                elif isFlori(s["text"], face): #is floraison
                    lis = s["text"].split()[1:] #remove first part
                    sent = s["text"]
                    sent = sent[sent.find(" ") + 1:] #text after first space
                    floriCount += 1
                    if lis:
                        if "-" in lis[0]: #split according to date
                            months = lis[0].split("-")
                            frame.iloc[-1]['FloraisonStart'] = months[0]
                            frame.iloc[-1]['FloraisonEnd'] = months[1][:-1]
                            lis = lis[1:]
                            sent = sent[sent.find(" ") + 1:]
                        floriCount += 1
                    if lis:
                        frame.iloc[-1]['FloraisonNotes'] = sent
                        floriCount += 1

                    
                    
                    
# print(item)
# print(arr)
frame

,Family,GenusDescription,Genus,epithet,author,syn.,Plate,description,FloraisonStart,FloraisonEnd,FloraisonNotes,L.,S.,Aire géogr.,Notes
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,Arbutus,unedo,0,0,0,0,0,0,"et Broummana (P, Vt, Bnm, Mt, Pb).",0,0,Aire géogr. — Tour de la Méditerranée européen...,0
2,0,0,Arbutus,andrachne,0,0,0,0,février,avril,Dagh (Pb).,0,0,"Aire géogr. — Méditerranée orientale, Crimée, ...",0
3,0,0,Samolus,valerandi,0,0,0,0,mai,juillet,"Bord de suintements constants, d'eau douce ou...","L. Ct. Saïda (Bl), Beyrouth (Vt), Nahr-el-Kelb...",S. Ct. Lattaquié (P). Mi. Bhamra (Har). Mm. So...,Aire géogr. — Subcosmique.céolées. Calice pent...,0
4,0,LYSIMACHIA L. — Plantes annuelles ou vivaces. ...,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,0,0,Andryala,integrifolia,0,0,0,0,avril,juin,Peut-être naturalisé à partir de graines venue...,0,0,"Aire géogr. — Sicile, Italie méridionale, Afri...",0
1043,0,0,Hieracium,bauhinii,0,0,0,0,mai,août,Slenfé (Pb).,0,0,"Aire géogr. — Europe moyenne et méridionale, G...",0
1044,0,0,Hieracium,procerum,0,0,0,0,0,0,été. Lieux humides en montagne.,"L. Mm. 'Ain Foha au-dessus de Bcharré (Ky), La...",0,"Aire géogr. — Caucase, Turquie, Liban, Iran, A...",0
1045,0,0,Hieracium,schmidtii,0,0,0,0,juin,août,Terrains rocheux en montagne.,"L. Liban (Lab.). Mm. Jourd Hasroun (Bl), au-de...",0,"Aire géogr. — Turquie, Liban.",0


In [51]:
frame.to_csv("notes5.csv")

In [30]:
df = pd.read_excel('text_to_spreadsheet_template.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'text_to_spreadsheet_template.xlsx'